<a href="https://colab.research.google.com/github/ShalokBalodi/Sentiment-analysis-/blob/main/NLP_Sentiment_model_basic_layout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Import our libraries (this may take a minute or two)
import pandas as pd   # Great for tables (google spreadsheets, microsoft excel, csv).
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import spacy
import wordcloud
import os # Good for navigating your computer's files
import sys
pd.options.mode.chained_assignment = None #suppress warnings

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
nltk.download('wordnet')
nltk.download('punkt')

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!python -m spacy download en_core_web_md
import en_core_web_md
text_to_nlp = spacy.load('en_core_web_md')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2023-11-08 06:46:17.852542: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 06:46:17.852589: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 06:46:17.852626: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-08 06:46:19.511256: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [148]:
data=pd.read_excel('/content/NLP_dataset.xlsx')
data.head()


,text,stars
0,These lights are great; I like the longer adap...,5.0
1,This is another LED strip light that has a lot...,5.0
2,These lights are awesome! I have them in all 4...,5.0
3,This was even better than I imagined. My 10-ye...,5.0
4,An excellent product. Work properly.,5.0


In [149]:

len(data)

763

In [150]:
def is_good_review(num_stars):
    if  num_stars>3:
        return 1
    elif num_stars==3:
        return 0
    else:
      return -1
# Change the stars column to either be 'good' or 'bad'.
data['is_good_review'] = data['stars'].apply(is_good_review)
data.head()

,text,stars,is_good_review
0,These lights are great; I like the longer adap...,5.0,1
1,This is another LED strip light that has a lot...,5.0,1
2,These lights are awesome! I have them in all 4...,5.0,1
3,This was even better than I imagined. My 10-ye...,5.0,1
4,An excellent product. Work properly.,5.0,1


In [151]:
X_text = data['text']
y = data['is_good_review']

#text cleaning function
def tokenize(text):
    clean_tokens = []
    for token in text_to_nlp(text):
        if (not token.is_stop) & (token.lemma_ != '-PRON-') & (not token.is_punct): # -PRON- is a special all inclusive "lemma" spaCy uses for any pronoun, we want to exclude these
            clean_tokens.append(token.lemma_)
    return clean_tokens

In [152]:
X_text=data['text']
y=data['is_good_review']
bow_transformer = CountVectorizer(analyzer=tokenize, max_features=2500).fit(X_text)
bow_transformer.vocabulary_
len(bow_transformer.vocabulary_)
X = bow_transformer.transform(X_text)
pd.DataFrame(X.toarray())
#NOW WE ARE READY TO BUILD A MODEL


,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
759,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
760,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
761,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [165]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=53)
 logistic_model =LogisticRegression()
 logistic_model.fit(X_train,y_train)
 pred=logistic_model.predict(X_test)
 accuracy = accuracy_score(pred,y_test)
 print (accuracy)# 53 92 81 82

0.8461538461538461


In [159]:

#@title Enter a review to see your model's classification
example_review = input()
prediction = logistic_model.predict(bow_transformer.transform([example_review]))
print(prediction)
if prediction==1:
  print ("This was a GOOD review!")
elif prediction==-1:
  print ("This was a BAD review!")
else:
  print("Neutral")


average
[1]
This was a GOOD review!


In [155]:
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.04, random_state=18)
model = MultinomialNB()
model.fit(X_train,y_train)
pred=model.predict(X_test)
ac=accuracy_score(pred,y_test)
print(ac)

0.8387096774193549


In [136]:
example_review =input()
prediction = model.predict(bow_transformer.transform([example_review]))
print(prediction)
if prediction==1:
  print ("This was a GOOD review!")
elif prediction==-1:
  print ("This was a BAD review!")
else:
  print("Neutral")


mediocre
[0]
Neutral


In [ ]:
#@title Enter a review to see your model's classification
example_review = input()
prediction = logistic_model.predict(bow_transformer.transform([example_review]))
print(prediction)
if prediction==1:
  print ("This was a GOOD review!")
elif prediction==-1:
  print ("This was a BAD review!")
else:
  print("Neutral")



I liked this product but only one set worked out of the two sets. I just received another box of two today but I would like to send it back.
[1]
This was a GOOD review!


In [ ]:
#@title Enter a review to see your model's classification
example_review = input()
prediction = logistic_model.predict(bow_transformer.transform([example_review]))
print(prediction)
if prediction==1:
  print ("This was a GOOD review!")
elif prediction==-1:
  print ("This was a BAD review!")
else:
  print("Neutral")



First order one set of lights worked, sent back and replaced second set one of the strands just went out they worked for about a month and a half
[-1]
This was a BAD review!


In [156]:
def tokenize_vecs(text):
    clean_tokens = []
    for token in text_to_nlp(text):
        if (not token.is_stop) & (token.lemma_ != '-PRON-') & (not token.is_punct):
          # -PRON- is a special all inclusive "lemma" spaCy uses for any pronoun, we want to exclude these
            clean_tokens.append(token)
    return clean_tokens

In [157]:
X_word2vec = []
for text in X_text:
  review = tokenize_vecs(text) # returns cleaned list of spacy tokens
  review_vec = [0]*300
  for word in review:
    review_vec += word.vector
  review_vec = review_vec / len(review)
  X_word2vec.append(review_vec)

X_word2vec = np.array(X_word2vec)


In [158]:
w2v_model = LogisticRegression(max_iter = 3000)
# train-test split
for i in range(1,100):
 X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(X_word2vec, y, test_size=0.075, random_state=i)
 w2v_model.fit(X_train_word2vec, y_train_word2vec)
 w2v_preds = w2v_model.predict(X_test_word2vec)
 accuracy = accuracy_score(y_test_word2vec, w2v_preds)
 print (i,accuracy)

1 0.7068965517241379
2 0.6724137931034483
3 0.6551724137931034
4 0.6206896551724138
5 0.603448275862069
6 0.6896551724137931
7 0.603448275862069
8 0.6206896551724138
9 0.5862068965517241
10 0.7068965517241379
11 0.6551724137931034
12 0.6896551724137931
13 0.7413793103448276
14 0.6896551724137931
15 0.5517241379310345
16 0.6896551724137931
17 0.6724137931034483
18 0.7241379310344828
19 0.5689655172413793
20 0.6896551724137931
21 0.7586206896551724
22 0.6724137931034483
23 0.6724137931034483
24 0.7068965517241379
25 0.7241379310344828
26 0.7068965517241379
27 0.6724137931034483
28 0.7068965517241379
29 0.6551724137931034
30 0.6206896551724138
31 0.6896551724137931
32 0.6206896551724138
33 0.6379310344827587
34 0.6724137931034483
35 0.5517241379310345
36 0.4827586206896552
37 0.6896551724137931
38 0.7586206896551724
39 0.7068965517241379
40 0.603448275862069
41 0.6551724137931034
42 0.7068965517241379
43 0.6206896551724138
44 0.7241379310344828
45 0.7241379310344828
46 0.7068965517241379


In [147]:
X_text=[input()]
X_word2vec = []
for text in X_text:
  review = tokenize_vecs(text) # returns cleaned list of spacy tokens
  review_vec = [0]*300
  for word in review:
    review_vec += word.vector
  review_vec = review_vec / len(review)
  X_word2vec.append(review_vec)

X_word2vec = np.array(X_word2vec)
print(X_word2vec)

v
[[-1.10030003e+01 -4.28520012e+00  1.02580004e+01 -9.02989984e-01
  -4.03830004e+00 -1.37470007e-01 -9.95400012e-01 -3.34229994e+00
  -4.19369984e+00 -3.32150006e+00  4.04839993e+00  7.40329981e-01
  -2.73679996e+00 -5.36079979e+00 -9.05560017e+00 -1.44089997e+00
  -1.36340007e-01  1.15030003e+01  5.44710016e+00  5.90419993e-02
   3.62389994e+00 -4.10459995e+00 -1.14580002e+01  1.49460006e+00
  -9.51939964e+00 -9.21739960e+00  8.37940025e+00  5.53889990e+00
   4.71810007e+00 -1.47430003e+00 -2.25469995e+00  1.25049993e-01
   4.53340006e+00 -4.97979999e-01  5.25040007e+00 -2.45079994e+00
   9.10099983e-01  2.76830006e+00 -1.01610003e+01 -7.69290018e+00
  -5.49590015e+00  6.13990016e-02  2.83349991e+00 -4.91270018e+00
  -1.23829998e-01  5.75519991e+00  4.52099991e+00  6.92479992e+00
   2.65009999e+00  3.04650009e-01  4.59350014e+00 -5.62709999e+00
  -7.64249992e+00  3.93540001e+00 -6.53049994e+00  8.55819988e+00
   7.19150019e+00  2.22890005e-02  1.13000000e+00 -4.09880018e+00
  -3.186

In [120]:
w2v_preds = w2v_model.predict(X_word2vec)
print(w2v_preds)



[-1]
